## Linear and Polynomial Regression for Skyrmion trajectories prediction

**Imports**

In [58]:
# Standard imports
import pandas as pd
import numpy as np

from tqdm import tqdm  # for progress bar

**Read the data**

In [62]:
directory = 'Rec_EDGE_300K_1L_50MA.out'

data = pd.read_csv(directory + '/trajectories.csv')
data.head()

,Unnamed: 0,y,x,mass,size,ecc,signal,raw_mass,ep,frame,particle
0,0,24.420047,61.809992,33.895088,3.785002,0.048289,0.451671,91.250977,0.000493,0,0
1,1,31.518261,109.009463,33.850101,3.792741,0.067499,0.455270,91.282349,0.000493,0,1
2,2,51.658864,41.007417,34.208199,3.811746,0.062159,0.449871,92.517654,0.000486,0,2
3,3,60.994689,82.173861,34.559098,3.818268,0.046138,0.455270,93.368622,0.000482,0,3
4,4,61.572998,129.252586,33.747531,3.807508,0.059245,0.453470,91.835289,0.000490,0,4


**Drop the unused columns**

In [63]:
unused_columns = ['Unnamed: 0', 'mass', 'size', 'ecc', 'signal', 'raw_mass', 'ep']

data = data.drop(columns=unused_columns)
data.head()

,y,x,frame,particle
0,24.420047,61.809992,0,0
1,31.518261,109.009463,0,1
2,51.658864,41.007417,0,2
3,60.994689,82.173861,0,3
4,61.572998,129.252586,0,4


**Fill in missing values with average positions (if a skyrmion is missing for more than one frame, it might not be very precise, but it should not be a big issue here)**

In [64]:
no_skyrmions = data[data['frame'] == 0].shape[0]
no_skyrmions

15

In [76]:
# ids of initial particles
ids = list(range(no_skyrmions))

# iterate through the frames
for f in tqdm(data['frame'].unique()):
    for p in range(no_skyrmions):
        # this means the skyrmion p is missing in frame f
        if not any(data[data['frame'] == f]['particle'] == p):
            
            # find previous coorinates
            x_prev = data[(data['frame'] == f-1) & (data['particle'] == p)]['x'].values[0]
            y_prev = data[(data['frame'] == f-1) & (data['particle'] == p)]['y'].values[0]
            
            #find next coordinates
            for next_frame in range((f+1).astype(int), len(data['frame'].unique())):
                if any(data[data['frame'] == f]['particle'] == p):
                    x_next = data[(data['frame'] == next_frame) & (data['particle'] == p)]['x'].values[0]
                    y_next = data[(data['frame'] == next_frame) & (data['particle'] == p)]['y'].values[0]
                    break
                    
            # new coordinates
            x_new = (x_prev + x_next) / 2
            y_new = (y_prev + y_next) / 2
            
            data = data.append({'y' : y_new,
                                'x' : x_new,
                                'frame' : f,
                                'particle': p}, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:14<00:00, 54.29it/s]


(12000, 4)

In [87]:
data = data.sort_values(by=['frame', 'particle'])

In [88]:
data

,y,x,frame,particle
0,24.420047,61.809992,0.0,0.0
3,31.518261,109.009463,0.0,1.0
4,51.658864,41.007417,0.0,2.0
7,60.994689,82.173861,0.0,3.0
1,61.572998,129.252586,0.0,4.0
...,...,...,...,...
11924,26.838018,9158.734705,799.0,10.0
11927,92.153535,8909.539660,799.0,11.0
11923,160.675052,8812.401110,799.0,12.0
11932,156.663224,8600.194927,799.0,13.0


**Check that there are no more missing values**

In [89]:
for f in tqdm(data['frame'].unique()):
    if (data[data['frame'] == f]['particle'].shape[0] < no_skyrmions):
        print(f)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:00<00:00, 2905.98it/s]


**Format the data so that it is in the format (frame, next_frame)**

In [90]:
frames = []

# iterate through the frames
for f in tqdm(data['frame'].unique()):
    coordinates = None
    for p in data[data['frame'] == f]['particle']:
        particle = data[(data['frame'] == f) & (data['particle'] == p)]
        coordinates = np.append(coordinates, [particle['x'].values[0], particle['y'].values[0]]) if coordinates is not None else [particle['x'].values[0], particle['y'].values[0]]
    frames.append(list(coordinates))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 800/800 [00:06<00:00, 120.79it/s]


**Place data in DataFrame**

In [91]:
# data in columns ['frame', 'next_frame']
df = pd.DataFrame(columns=['frame', 'next_frame'])

for i in range(1, len(frames)):
    df = df.append({'frame': frames[i-1], 'next_frame': frames[i]}, ignore_index=True)

df

,frame,next_frame
0,"[61.80999150562753, 24.420046719048628, 109.00...","[70.80009877085162, 22.786106233538195, 115.03..."
1,"[70.80009877085162, 22.786106233538195, 115.03...","[78.53846153846153, 23.79722075869336, 130.692..."
2,"[78.53846153846153, 23.79722075869336, 130.692...","[92.43681939593179, 22.50037668652832, 141.306..."
3,"[92.43681939593179, 22.50037668652832, 141.306...","[104.59782115297321, 22.04513325984048, 153.67..."
4,"[104.59782115297321, 22.04513325984048, 153.67...","[114.654700661428, 17.817739838317603, 167.487..."
...,...,...
794,"[8971.234802590348, 161.42225471763803, 8889.3...","[8981.526291116494, 162.7594343308071, 8905.46..."
795,"[8981.526291116494, 162.7594343308071, 8905.46...","[8990.157367074604, 162.0370600843532, 8920.93..."
796,"[8990.157367074604, 162.0370600843532, 8920.93...","[9007.540353356892, 162.55978798586568, 8926.0..."
797,"[9007.540353356892, 162.55978798586568, 8926.0...","[9017.586395147311, 165.976863084922, 8941.781..."


**Split data for training and testing**

In [92]:
from sklearn.model_selection import train_test_split

X = df['frame'].tolist()
y = df['next_frame'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

**Train model with Linear Regression**

In [93]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression()

**Model evaluation**

In [99]:
from sklearn.metrics import mean_squared_error, r2_score

# model evaluation for training set
y_train_predict = lin_model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
r2 = r2_score(y_train, y_train_predict)

print("The model performance for training set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))
print("\n")

# model evaluation for testing set
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_test_predict)))
r2 = r2_score(y_test, y_test_predict)

print("The model performance for testing set")
print("--------------------------------------")
print('RMSE is {}'.format(rmse))
print('R2 score is {}'.format(r2))

The model performance for training set
--------------------------------------
RMSE is 114.45948435910825
R2 score is 0.9783630031403245


The model performance for testing set
--------------------------------------
RMSE is 199.73416702118666
R2 score is 0.9663727071319047


In [108]:
print(y_test[100] - y_test_predict[100])

[ 30.90086991  -3.23040637  41.09280145  -4.03078131  29.36102736
   1.48010687 -12.60024849   2.20631749  45.55601372   0.19877135
 -10.97073678   1.71114222 -13.79354006  -6.70909065   1.17760844
   8.43691993  15.1093166   -1.34147994  41.34113692   2.60119802
 -54.69217387  -2.16762231 -39.58241282   3.55239605 -12.03001087
   4.84036499  13.2824046    0.8145066   -0.47842478  -0.08533784]


**Train with Polynoial Regression**

In [95]:
from sklearn.preprocessing import PolynomialFeatures

def create_polynomial_regression_model(degree):
    poly_features = PolynomialFeatures(degree=degree)

    # transform the features to higher degree features.
    X_train_poly = poly_features.fit_transform(X_train)

    # fit the transformed features to Linear Regression
    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train)

    # predicting on training data-set
    y_train_predicted = poly_model.predict(X_train_poly)

    # predicting on test data-set
    y_test_predict = poly_model.predict(poly_features.fit_transform(X_test))

    # evaluating the model on training dataset
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_predicted))
    r2_train = r2_score(y_train, y_train_predicted)

    # evaluating the model on test dataset
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
    r2_test = r2_score(y_test, y_test_predict)

    print("The model performance for the training set")
    print("-------------------------------------------")
    print("RMSE of training set is {}".format(rmse_train))
    print("R2 score of training set is {}".format(r2_train))

    print("\n")

    print("The model performance for the test set")
    print("-------------------------------------------")
    print("RMSE of test set is {}".format(rmse_test))
    print("R2 score of test set is {}".format(r2_test))

In [105]:
create_polynomial_regression_model(2)

The model performance for the training set
-------------------------------------------
RMSE of training set is 39.58333827868031
R2 score of training set is 0.996683528827107


The model performance for the test set
-------------------------------------------
RMSE of test set is 140219.76073076884
R2 score of test set is -51662.79270901341


In [104]:
create_polynomial_regression_model(3)

The model performance for the training set
-------------------------------------------
RMSE of training set is 2.955144771062068e-07
R2 score of training set is 1.0


The model performance for the test set
-------------------------------------------
RMSE of test set is 446956.3628086503
R2 score of test set is -922769.729560856
